In [ ]:
# https://leetcode.com/problems/lfu-cache/description/

In [ ]:
# https://www.youtube.com/watch?v=0PSB9y8ehbk

In [1]:


class ListNode:
    def __init__(self, key=0, val=0, freq=1, prev=None, next=None):
        self.key = key
        self.val = val
        self.prev = prev
        self.next = next
        self.freq = freq


class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.head = ListNode(-1,-1)
        self.tail = ListNode(-1,-1) 
        self.head.next = self.tail
        self.tail.prev = self.head
        self.mp = {}

    def get(self, key: int) -> int:
        if key in self.mp:
            node = self.mp[key]
            val = node.val
            freq = node.freq
            if node != self.head.next:
                self.put(key, val, freq)
            return val
        return -1
            
    def pop(self, key = None):
        if key == None:
            # remove the node before the tail
            second_last = self.tail.prev
            second_last.prev.next = second_last.next
            second_last.next.prev = second_last.prev
            self.mp.pop(second_last.key)
            return second_last
        else:
            temp = self.mp[key]
            temp.prev.next = temp.next
            temp.next.prev = temp.prev
            self.mp.pop(temp.key)
            return temp

    def put(self, key: int, val: int, freq: int) -> None:
        if key not in self.mp:
            if len(self.mp) < self.capacity:
                # if node is not present add it after head
                node = ListNode(key, val, freq)
                self.head.next.prev = node
                node.next = self.head.next
                node.prev = self.head
                self.head.next = node

                self.mp[key] = node
                return node
            else:
                self.pop()
                return self.put(key, val, freq)
        else:
            node = self.mp[key]
            if self.head.next == node:
                # if node is after head
                # then we just need to make sure that val property of that node is the val given in put method
                node.val = val
                return node
            else:
                self.pop(key)
                return self.put(key, val, freq)                


class LFUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.mp = {}
        self.freq = {}

    def get(self, key: int) -> int:
        if key in self.mp:
            val = self.mp[key].val
            freq = self.mp[key].freq
            self.freq[freq].pop(key)

            if len(self.freq[freq].mp) == 0:
                self.freq.pop(freq)
            if freq + 1 not in self.freq:
                  self.freq[freq + 1] = LRUCache(self.capacity)

            self.mp[key] = self.freq[freq + 1].put(key, val, freq + 1)
            return val
        return -1

    def put(self, key: int, val: int) -> None:
        if key not in self.mp:
            if len(self.mp) < self.capacity:
                if 1 not in self.freq:
                    self.freq[1] = LRUCache(self.capacity)
                self.mp[key] = self.freq[1].put(key, val, 1)
            else:
                lowest_freq = min(self.freq.keys())
                removed = self.freq[lowest_freq].pop()
                if len(self.freq[lowest_freq].mp) == 0:
                    self.freq.pop(lowest_freq)
                self.mp.pop(removed.key)
                self.put(key, val)
        else:
            # if node with existing key comes, we will update it's val to new val and increase it's frequency by 1
            node = self.mp[key]
            freq = node.freq
            self.freq[freq].pop(key)
            if len(self.freq[freq].mp) == 0:
                self.freq.pop(freq)
            if freq + 1 not in self.freq:
                  self.freq[freq + 1] = LRUCache(self.capacity)
            self.mp[key] = self.freq[freq + 1].put(key, val, freq + 1)
            


# Your LFUCache object will be instantiated and called as such:
# obj = LFUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,val)